In [1]:
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler

In [ ]:
# Load dataset
data = pd.read_csv("fraudTest.csv")

# Quick check
data.head()


,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,2020-06-21 12:14:25,2291163933867244,fraud_Kirlin and Sons,personal_care,2.86,Jeff,Elliott,M,351 Darlene Green,Columbia,...,33.9659,-80.9355,333497,Mechanical engineer,1968-03-19,2da90c7d74bd46a0caf3777415b3ebd3,1371816865,33.986391,-81.200714,0
1,2020-06-21 12:14:33,3573030041201292,fraud_Sporer-Keebler,personal_care,29.84,Joanne,Williams,F,3638 Marsh Union,Altonah,...,40.3207,-110.4360,302,"Sales professional, IT",1990-01-17,324cc204407e99f51b0d6ca0055005e7,1371816873,39.450498,-109.960431,0
2,2020-06-21 12:14:53,3598215285024754,"fraud_Swaniawski, Nitzsche and Welch",health_fitness,41.28,Ashley,Lopez,F,9333 Valentine Point,Bellmore,...,40.6729,-73.5365,34496,"Librarian, public",1970-10-21,c81755dbbbea9d5c77f094348a7579be,1371816893,40.495810,-74.196111,0
3,2020-06-21 12:15:15,3591919803438423,fraud_Haley Group,misc_pos,60.05,Brian,Williams,M,32941 Krystal Mill Apt. 552,Titusville,...,28.5697,-80.8191,54767,Set designer,1987-07-25,2159175b9efe66dc301f149d3d5abf8c,1371816915,28.812398,-80.883061,0
4,2020-06-21 12:15:17,3526826139003047,fraud_Johnston-Casper,travel,3.19,Nathan,Massey,M,5783 Evan Roads Apt. 465,Falmouth,...,44.2529,-85.0170,1126,Furniture designer,1955-07-06,57ff021bd3f328f8738bb535c302a31b,1371816917,44.959148,-85.884734,0


In [3]:
# Drop columns with too many categories or not useful
data = data.drop(columns=[
    "cc_num", "first", "last", "street",
    "trans_num", "trans_date_trans_time",
    "job", "merchant"
])


In [ ]:
# Label encode gender
le_gender = LabelEncoder()
data['gender'] = le_gender.fit_transform(data['gender'])

# One-hot encode 'category' and 'state'
ohe = OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore')

cat_features = data[['category', 'state']]

ohe_encoded = ohe.fit_transform(data[cat_features])

# Convert back to DataFrame with proper column names
ohe_df = pd.DataFrame(
    ohe_encoded,
    columns=ohe.get_feature_names_out(['category', 'state']),
    index=data.index
)

# Concatenate back to dataset and drop originals
data = pd.concat([data.drop(columns=['category', 'state']), ohe_df], axis=1)
data = data.drop(columns=['Unnamed: 0', 'city', 'zip', 'dob'])